In [ ]:
def quantile_strategie(column, row, summary):
    try:
        if row[column] == summary[column]['min']:
            return 'min'
        elif row[column] <= summary[column]['25%']:
            return '25%'
        elif row[column] <= summary[column]['50%']:
            return '50%'
        elif row[column] <= summary[column]['75%']:
            return '75%'
        else:
            return 'max'
    except KeyError:
        return row[column]

In [ ]:
gray_color = [128, 128, 128, 160]
blue_color = [3, 169, 244, 160]
green_color = [92, 176, 81, 160]
yellow_color = [255, 255, 0, 160]
red_color = [255, 0, 0, 160]

inner_style_dictionary = {
    'min': {
        'fillColor': gray_color,
        'lineColor': gray_color,
        'lineWidth': 5
        # 'dashArray': [1, 1]
    },
    '25%': {
        'fillColor': blue_color,
        'lineColor': blue_color,
        'lineWidth': 10
        # 'dashArray': [1, 1]
    },
    '50%': {
        'fillColor': green_color,
        'lineColor': green_color,
        'lineWidth': 15
        # 'dashArray': [1, 1]
    },
    '75%': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
    'max': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
}